# TFM Suministro Global - Sector de Actividad NACE2- Aplicación/usos de Material  
# _Neo4j - py2neo_


## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
https://nacev2.com/es   
https://ec.europa.eu/competition/mergers/cases/index/nace_all.html

### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [2]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [3]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [4]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [5]:
#que es el objeto - tx
tx = graph_test01.begin()

In [6]:
tx

In [11]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS PREVIOS - 
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, Ntotal)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88


## CARGAR DATOS

In [10]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [12]:
#DATOS: Fichero de SECTORES DE ACTIVIDAD vinculados a APLICACIONES-USOS de materiales de suministro crítico GLOBAL
#        -- Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLSectorAplic_Cargar.csv
#CONTENIDO: Sectores NACE2 abreviados (34 chars) de actividad vinculables a los Grupo de Aplicación/uso de cada material 
#USO: Crear los NODOS de tipo "SectorAplicacion" asociable a nodos "GrupoAplicacion", agrupando 1 sector N nodos GrupoAplicacion
#     De este modo a partir de cada Sector NACE2 oficial se puede investigar toda la cadena de uso-materiales- producción
## -- propiedades de cada Sector: nombre (name)  y Valor Añadido del sector (ValorAnadidoSector)
## -- los datos de suministro -posteriores fijarán puede relaciones entre materiales y sus grupos de uso/aplicación
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLSectorAplic_Cargar.csv" 
dfSector = pd.read_csv(rutaFichero, sep=';')
#dataset cargado

In [14]:
dfSector.head()

,SectorAplicacion,ValorAnadidoSector
0,"B06-Extraction (crude, gas)**","19,75"
1,B09-Mining (support)**,"3,40"
2,C10-food prods.,"155,88"
3,C11-beverages,"32,51"
4,"C16-Wood,Cork,Straw,Plaiting-NoFur","27,97"


In [15]:
dfSector.tail()

,SectorAplicacion,ValorAnadidoSector
20,C31-furniture,"26,17"
21,C32-Other manuf.**,"39,16"
22,C33-Repair&install.Machinery-Equip,"52,33"
23,E39-Remed.Activs &Waste MngSrvs,"1,30"
24,Z99-Others**,"0,00"


In [16]:
dfSector.shape

(25, 2)

In [17]:
dfSector.columns

Index(['SectorAplicacion', 'ValorAnadidoSector'], dtype='object')

In [18]:
nFilas = dfSector.shape[0]

In [19]:
nFilas

25

In [21]:
### Cargados en memoria los 25 sectores NACE2 de actividad vinculables a cada grupo de aplicación/uso de material en la 
### cadena de suministro global

# PREVISUALIZACION DE DATOS - USOS


In [38]:
for i in range(0, nFilas):
    print(dfSector.SectorAplicacion[i])
    print(dfSector.ValorAnadidoSector[i])
#cantidades en millones de euros

B06-Extraction (crude, gas)**
19,75
B09-Mining (support)**
3,40
C10-food prods. 
155,88
C11-beverages 
32,51
C16-Wood,Cork,Straw,Plaiting-NoFur
27,97
C17-paper & paper prods. 
38,91
C19-Coke & Refined Petrol. Ps
17,29
C20-Chemicals & Chemc.Prods.
105,51
C21-BasicPharmaceutic.&PhPreps
80,18
C22-rubber & plastic prods. 
75,98
C23-Oth.NoMetal.mineral Prods.
57,26
C24 - Manufacture of basic metals
55,43
C24-basic metals 
55,43
C25-fabricated metal prods., 
148,35
C25-FabrMetalProds-NoMachin-Eq
148,35
C26-Computer,Electron.Optic.Ps
65,70
C27-electrical equipment 
80,75
C28-Machinery &Equipment n.e.c
182,59
C29-MotorVehicle,Trailer&SemiT
160,60
C30-other transport equipment 
44,30
C31-furniture 
26,17
C32-Other manuf.**
39,16
C33-Repair&install.Machinery-Equip
52,33
E39-Remed.Activs &Waste MngSrvs
1,30
Z99-Others**
0,00


In [34]:
# columna cero identifica el tipo de nodo - etiqueta Neo4j- a crear
dfSector.columns[0]

'SectorAplicacion'

In [39]:
#  Crear NODO DE SECTOR DE APLICACION - por cada fila junto con sus PROPIEDADES
#  Nodos: SectorAplicacion -- para agrupar con código oficial NACE2 las aplicaciones de cada material crítico global
# IMPORTANTE.- La representación decimal en Neo2j se fija a punto ".",  reemplazándose la coma "," original
TipoNodo = " :SectorAplicacion " 
for i in range(0, nFilas):
    strQry = "CREATE (s " + TipoNodo                                           \
        + "{"                                                                   \
        + "name: '" + dfSector.SectorAplicacion[i]                              \
        + "', "                                                                 \
        + "ValorAnadidoSector: " + dfSector.ValorAnadidoSector[i]               \
        + " })"                                                                 \
        + " RETURN s" 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas

QRY [0]> CREATE (s  :SectorAplicacion {name: 'B06-Extraction (crude, gas)**', ValorAnadidoSector: 19,75 }) RETURN s
QRY [1]> CREATE (s  :SectorAplicacion {name: 'B09-Mining (support)**', ValorAnadidoSector: 3,40 }) RETURN s
QRY [2]> CREATE (s  :SectorAplicacion {name: 'C10-food prods. ', ValorAnadidoSector: 155,88 }) RETURN s
QRY [3]> CREATE (s  :SectorAplicacion {name: 'C11-beverages ', ValorAnadidoSector: 32,51 }) RETURN s
QRY [4]> CREATE (s  :SectorAplicacion {name: 'C16-Wood,Cork,Straw,Plaiting-NoFur', ValorAnadidoSector: 27,97 }) RETURN s
QRY [5]> CREATE (s  :SectorAplicacion {name: 'C17-paper & paper prods. ', ValorAnadidoSector: 38,91 }) RETURN s
QRY [6]> CREATE (s  :SectorAplicacion {name: 'C19-Coke & Refined Petrol. Ps', ValorAnadidoSector: 17,29 }) RETURN s
QRY [7]> CREATE (s  :SectorAplicacion {name: 'C20-Chemicals & Chemc.Prods.', ValorAnadidoSector: 105,51 }) RETURN s
QRY [8]> CREATE (s  :SectorAplicacion {name: 'C21-BasicPharmaceutic.&PhPreps', ValorAnadidoSector: 80,18 }

In [26]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')

In [40]:
# Crear un nodo por cada fila con sus propiedades
#nodos de GRUPOAPLICACION
## query para crea nodo con sus  propiedades
TipoNodo = " :SectorAplicacion " 
for i in range(0, nFilas):
    strQry = "CREATE (s " + TipoNodo                                           \
        + "{"                                                                   \
        + "name: '" + dfSector.SectorAplicacion[i]                              \
        + "', "                                                                 \
        + "ValorAnadidoSector: " + dfSector.ValorAnadidoSector[i].replace(',', '.')               \
        + " })"                                                                 \
        + " RETURN s" 
#ejecutar Query en BD abierta - Neo4j
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> CREATE (s  :SectorAplicacion {name: 'B06-Extraction (crude, gas)**', ValorAnadidoSector: 19.75 }) RETURN s
EJECUTADA QRY [1]> CREATE (s  :SectorAplicacion {name: 'B09-Mining (support)**', ValorAnadidoSector: 3.40 }) RETURN s
EJECUTADA QRY [2]> CREATE (s  :SectorAplicacion {name: 'C10-food prods. ', ValorAnadidoSector: 155.88 }) RETURN s
EJECUTADA QRY [3]> CREATE (s  :SectorAplicacion {name: 'C11-beverages ', ValorAnadidoSector: 32.51 }) RETURN s
EJECUTADA QRY [4]> CREATE (s  :SectorAplicacion {name: 'C16-Wood,Cork,Straw,Plaiting-NoFur', ValorAnadidoSector: 27.97 }) RETURN s
EJECUTADA QRY [5]> CREATE (s  :SectorAplicacion {name: 'C17-paper & paper prods. ', ValorAnadidoSector: 38.91 }) RETURN s
EJECUTADA QRY [6]> CREATE (s  :SectorAplicacion {name: 'C19-Coke & Refined Petrol. Ps', ValorAnadidoSector: 17.29 }) RETURN s
EJECUTADA QRY [7]> CREATE (s  :SectorAplicacion {name: 'C20-Chemicals & Chemc.Prods.', ValorAnadidoSector: 105.51 }) RETURN s
EJECUTADA QRY [8]> CREATE 

In [41]:
dfGraphMat= graph_test01.run("MATCH (n :SectorAplicacion ) RETURN n").data()

In [42]:
dfGraphMat[0]


{'n': Node('SectorAplicacion', ValorAnadidoSector=19.75, name='B06-Extraction (crude, gas)**')}

In [43]:
#validamos que todos los -nuevos- nodos se han creado :GrupoAplicacion   
graph_test01.run("MATCH (n :SectorAplicacion ) RETURN  LABELS(n), COUNT(n)")

LABELS(n),COUNT(n)
['SectorAplicacion'],25


In [46]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[4] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal"  
  
                                  
graph_test01.run(queryPRB).to_data_frame()



,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88
4,SectorAplicacion,25


# fin proceso - SECTORES NACE2 - CREADOS
**IMPORTES com PUNTO DECIMAL en BD Neo4j**

In [81]:
# fin archivo